# Run MATCH with PeTaL data

Created by Eric Kong on 21 June 2021.

In this notebook we run the MATCH algorithm (https://github.com/yuzhimanhua/MATCH) on Lens data labelled with PeTaL's taxonomy.

This notebook was originally run in Google Colaboratory with GPU acceleration.

In [1]:
import os

In [2]:
# If running on Google Colab: Mount drive and cd to workspace.
from google.colab import drive
drive.mount('/content/drive')

# NOTE: Replace DRIVE_PATH with the path you plan to clone MATCH into.
DRIVE_PATH = '/content/drive/Shareddrives/NASA'
%cd $DRIVE_PATH

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/NASA


In [3]:
!nvidia-smi

Fri Jun 25 18:17:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
if not os.path.exists('MATCH/'):
    !git clone https://github.com/yuzhimanhua/MATCH.git
else:
    print("You have already cloned the MATCH repository.")

You have already cloned the MATCH repository.


In [5]:
%cd ./MATCH
!ls

/content/drive/Shareddrives/NASA/MATCH
configure      main.py	     PeTaL-062414      run_models.sh
deepxml        MeSH	     predictions.txt   transform_data_PeTaL.py
evaluation.py  PeTaL	     preprocess.py     transform_data.py
joint	       PeTaL-062309  preprocess.sh
LICENSE        PeTaL-062312  README.md
MAG	       PeTaL-062315  requirements.txt


In [8]:
# Install requirements in requirements.txt
!chmod 755 -R .
!pip3 install -r requirements.txt

     |████████████████████████████████| 748.9MB 25kB/s 
     |████████████████████████████████| 8.8MB 7.3MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 133kB 56.4MB/s 
     |████████████████████████████████| 17.3MB 186kB/s 
     |████████████████████████████████| 24.8MB 1.5MB/s 
     |████████████████████████████████| 5.4MB 33.4MB/s 
     |████████████████████████████████| 24.2MB 52.4MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 286kB 53.8MB/s 
     |████████████████████████████████| 552kB 52.0MB/s 
  Created wheel for torchgpipe: filename=torchgpipe-0.0.5-cp37-none-any.whl size=37366 sha256=d472ee9a9749189e8e2a93485e6c8f46cf4efa911e2fcd210401e268acd881d9
  Stored in directory: /root/.cache/pip/wheels/3c/a9/48/2c05d34e5b2d6ac008536f5d940176fe0ea6b6dc792b97e121
Successfully built torchgpipe
ERROR: xarray 0.18.2 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is inc

In [6]:
DATASET = "PeTaL"
MODEL = "MATCH"

In [7]:
# Slightly modified preprocess.sh

!python3 transform_data_PeTaL.py --dataset $DATASET

!python preprocess.py \
--text-path {DATASET}/train_texts.txt \
--label-path {DATASET}/train_labels.txt \
--vocab-path {DATASET}/vocab.npy \
--emb-path {DATASET}/emb_init.npy \
--w2v-model {DATASET}/{DATASET}.joint.emb \

!python preprocess.py \
--text-path {DATASET}/test_texts.txt \
--label-path {DATASET}/test_labels.txt \
--vocab-path {DATASET}/vocab.npy \

[I 210625 18:18:07 preprocess:28] Vocab Size: 26834
[I 210625 18:18:07 preprocess:30] Getting Dataset: PeTaL/train_texts.txt Max Length: 500
[I 210625 18:18:07 preprocess:32] Size of Samples: 900
[I 210625 18:18:10 preprocess:28] Vocab Size: 26834
[I 210625 18:18:10 preprocess:30] Getting Dataset: PeTaL/test_texts.txt Max Length: 500
[I 210625 18:18:10 preprocess:32] Size of Samples: 100


In [ ]:
# Slightly modified run_models.sh

!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode train --reg 1
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode eval

!python evaluation.py \
--results {DATASET}/results/{MODEL}-{DATASET}-labels.npy \
--targets {DATASET}/test_labels.npy \
--train-labels {DATASET}/train_labels.npy

[I 210625 17:21:49 main:32] Model Name: MATCH
[I 210625 17:21:49 main:35] Loading Training and Validation Set
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'chemically_break_down_inorganic_compounds', 'detox/purify', 'manage_environmental_disturbances_in_a_community', 'protect_from_fire', 'protect_from_gases', 'send_vibratory_signals'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'detox/purify', 'protect_from_gases'] will be ignored
  .format(sorted(unknown, key=str)))
[I 210625 17:21:49 main:47] Number of Labels: 124
[I 210625 17:21:49 main:48] Size of Training Set: 800
[I 210625 17:21:49 main:49] Size of Validation Set: 100
[I 210625 17:21:49 main:66] Number of Edges: 101
[I 210625 17:21:49 main:68] Training
[I 210625 17:21:56 models:142] SWA Initial

## Ablation study: Effect of adding MAG and MeSH labels to text

Relevant to PeTaL Labeller Issue #53 (https://github.com/nasa-petal/PeTaL-labeller/issues/53).

Note: to turn on/off including MAG fields of study and MeSH terms I mucked about with the source file ./transform_data_PeTaL.py. For future it may be more convenient to add CLI options specifying such.

### Attempt with 1000 epochs, step = 10

In [ ]:
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/PeTaL.yaml --model-cnf configure/models/MATCH-PeTaL.yaml --mode train --reg 1

[I 210623 22:45:20 main:32] Model Name: MATCH
[I 210623 22:45:20 main:35] Loading Training and Validation Set
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'chemically_break_down_inorganic_compounds', 'detox/purify', 'manage_environmental_disturbances_in_a_community', 'protect_from_fire', 'protect_from_gases', 'send_vibratory_signals'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'detox/purify', 'protect_from_gases'] will be ignored
  .format(sorted(unknown, key=str)))
[I 210623 22:45:20 main:47] Number of Labels: 124
[I 210623 22:45:20 main:48] Size of Training Set: 800
[I 210623 22:45:20 main:49] Size of Validation Set: 100
[I 210623 22:45:20 main:66] Number of Edges: 101
[I 210623 22:45:20 main:68] Training
[I 210623 22:45:25 models:110] 2 512 train

In [ ]:
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/PeTaL.yaml --model-cnf configure/models/MATCH-PeTaL.yaml --mode eval

[I 210623 22:59:58 main:32] Model Name: MATCH
[I 210623 22:59:58 main:79] Loading Test Set
[I 210623 22:59:58 main:83] Size of Test Set: 100
[I 210623 22:59:58 main:85] Predicting
[I 210623 23:00:02 main:91] Finish Predicting


In [ ]:
!python evaluation.py --results PeTaL/results/MATCH-PeTaL-labels.npy --targets PeTaL/test_labels.npy --train-labels PeTaL/train_labels.npy

Precision@1,3,5: 0.61 0.49666666666666665 0.382
nDCG@1,3,5: 0.61 0.5242603369757589 0.5284999798212123


### Attempt with 1000 epochs, step = 100

In [ ]:
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/PeTaL.yaml --model-cnf configure/models/MATCH-PeTaL.yaml --mode train --reg 1

[I 210623 23:14:10 main:32] Model Name: MATCH
[I 210623 23:14:10 main:35] Loading Training and Validation Set
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'chemically_break_down_inorganic_compounds', 'detox/purify', 'manage_environmental_disturbances_in_a_community', 'protect_from_fire', 'protect_from_gases', 'send_vibratory_signals'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'detox/purify', 'protect_from_gases'] will be ignored
  .format(sorted(unknown, key=str)))
[I 210623 23:14:10 main:47] Number of Labels: 124
[I 210623 23:14:10 main:48] Size of Training Set: 800
[I 210623 23:14:10 main:49] Size of Validation Set: 100
[I 210623 23:14:10 main:66] Number of Edges: 101
[I 210623 23:14:10 main:68] Training
[I 210623 23:14:17 models:142] SWA Initial

In [ ]:
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/PeTaL.yaml --model-cnf configure/models/MATCH-PeTaL.yaml --mode eval

[I 210623 23:26:32 main:32] Model Name: MATCH
[I 210623 23:26:32 main:79] Loading Test Set
[I 210623 23:26:32 main:83] Size of Test Set: 100
[I 210623 23:26:32 main:85] Predicting
[I 210623 23:26:35 main:91] Finish Predicting


In [ ]:
!python evaluation.py --results PeTaL/results/MATCH-PeTaL-labels.npy --targets PeTaL/test_labels.npy --train-labels PeTaL/train_labels.npy

Precision@1,3,5: 0.67 0.49333333333333335 0.392
nDCG@1,3,5: 0.67 0.5408242591878821 0.5537992919593395


### Results with MAG labels, without MeSH labels

In [ ]:
# !cp -r PeTaL-062315 PeTaL

!python3 transform_data_PeTaL.py --dataset $DATASET

In [ ]:
!python preprocess.py \
--text-path {DATASET}/train_texts.txt \
--label-path {DATASET}/train_labels.txt \
--vocab-path {DATASET}/vocab.npy \
--emb-path {DATASET}/emb_init.npy \
--w2v-model {DATASET}/{DATASET}.joint.emb \

!python preprocess.py \
--text-path {DATASET}/test_texts.txt \
--label-path {DATASET}/test_labels.txt \
--vocab-path {DATASET}/vocab.npy \

[I 210623 23:51:54 preprocess:28] Vocab Size: 26834
[I 210623 23:51:54 preprocess:30] Getting Dataset: PeTaL/train_texts.txt Max Length: 500
[I 210623 23:51:54 preprocess:32] Size of Samples: 900
[I 210623 23:51:55 preprocess:28] Vocab Size: 26834
[I 210623 23:51:55 preprocess:30] Getting Dataset: PeTaL/test_texts.txt Max Length: 500
[I 210623 23:51:55 preprocess:32] Size of Samples: 100


In [ ]:
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode train --reg 1
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode eval

!python evaluation.py \
--results {DATASET}/results/{MODEL}-{DATASET}-labels.npy \
--targets {DATASET}/test_labels.npy \
--train-labels {DATASET}/train_labels.npy

[I 210623 23:53:26 main:32] Model Name: MATCH
[I 210623 23:53:26 main:35] Loading Training and Validation Set
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'chemically_break_down_inorganic_compounds', 'detox/purify', 'manage_environmental_disturbances_in_a_community', 'protect_from_fire', 'protect_from_gases', 'send_vibratory_signals'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'detox/purify', 'protect_from_gases'] will be ignored
  .format(sorted(unknown, key=str)))
[I 210623 23:53:26 main:47] Number of Labels: 124
[I 210623 23:53:26 main:48] Size of Training Set: 800
[I 210623 23:53:26 main:49] Size of Validation Set: 100
[I 210623 23:53:26 main:66] Number of Edges: 101
[I 210623 23:53:26 main:68] Training
[I 210623 23:53:32 models:142] SWA Initial

### Results without MAG labels, with MeSH labels

In [ ]:
!python3 transform_data_PeTaL.py --dataset $DATASET

In [ ]:
!python preprocess.py \
--text-path {DATASET}/train_texts.txt \
--label-path {DATASET}/train_labels.txt \
--vocab-path {DATASET}/vocab.npy \
--emb-path {DATASET}/emb_init.npy \
--w2v-model {DATASET}/{DATASET}.joint.emb \

!python preprocess.py \
--text-path {DATASET}/test_texts.txt \
--label-path {DATASET}/test_labels.txt \
--vocab-path {DATASET}/vocab.npy \

[I 210624 00:25:18 preprocess:28] Vocab Size: 26834
[I 210624 00:25:18 preprocess:30] Getting Dataset: PeTaL/train_texts.txt Max Length: 500
[I 210624 00:25:18 preprocess:32] Size of Samples: 900
[I 210624 00:25:19 preprocess:28] Vocab Size: 26834
[I 210624 00:25:19 preprocess:30] Getting Dataset: PeTaL/test_texts.txt Max Length: 500
[I 210624 00:25:19 preprocess:32] Size of Samples: 100


In [ ]:
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode train --reg 1
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode eval

!python evaluation.py \
--results {DATASET}/results/{MODEL}-{DATASET}-labels.npy \
--targets {DATASET}/test_labels.npy \
--train-labels {DATASET}/train_labels.npy

[I 210624 00:25:22 main:32] Model Name: MATCH
[I 210624 00:25:22 main:35] Loading Training and Validation Set
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'chemically_break_down_inorganic_compounds', 'detox/purify', 'manage_environmental_disturbances_in_a_community', 'protect_from_fire', 'protect_from_gases', 'send_vibratory_signals'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'detox/purify', 'protect_from_gases'] will be ignored
  .format(sorted(unknown, key=str)))
[I 210624 00:25:22 main:47] Number of Labels: 124
[I 210624 00:25:22 main:48] Size of Training Set: 800
[I 210624 00:25:22 main:49] Size of Validation Set: 100
[I 210624 00:25:22 main:66] Number of Edges: 101
[I 210624 00:25:22 main:68] Training
[I 210624 00:25:29 models:142] SWA Initial

### Results without MAG labels, without MeSH labels



In [ ]:
!python3 transform_data_PeTaL.py --dataset $DATASET

In [ ]:
!python preprocess.py \
--text-path {DATASET}/train_texts.txt \
--label-path {DATASET}/train_labels.txt \
--vocab-path {DATASET}/vocab.npy \
--emb-path {DATASET}/emb_init.npy \
--w2v-model {DATASET}/{DATASET}.joint.emb \

!python preprocess.py \
--text-path {DATASET}/test_texts.txt \
--label-path {DATASET}/test_labels.txt \
--vocab-path {DATASET}/vocab.npy \

Traceback (most recent call last):
  File "preprocess.py", line 8, in <module>
    from deepxml.data_utils import build_vocab, convert_to_binary
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 818, in get_code
  File "<frozen importlib._bootstrap_external>", line 917, in get_data
KeyboardInterrupt
Traceback (most recent call last):
  File "preprocess.py", line 8, in <module>
    from deepxml.data_utils import build_vocab, convert_to_binary
  File "/content/drive/Shareddrives/MATCH Attempt/MATCH/deepxml/data_utils.py", line 5, in <module>
    from sklearn.preprocessing import MultiLabelBinarizer, normalize
  File "/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/__init__.py", line 6, in 

In [ ]:
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode train --reg 1
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode eval

!python evaluation.py \
--results {DATASET}/results/{MODEL}-{DATASET}-labels.npy \
--targets {DATASET}/test_labels.npy \
--train-labels {DATASET}/train_labels.npy


Aborted!
[I 210624 21:33:03 main:32] Model Name: MATCH
[I 210624 21:33:03 main:79] Loading Test Set
[I 210624 21:33:03 main:83] Size of Test Set: 100
[I 210624 21:33:03 main:85] Predicting
[I 210624 21:33:06 main:91] Finish Predicting
Precision@1,3,5: 0.66 0.53 0.42
nDCG@1,3,5: 0.66 0.5620467187130749 0.5699775424270964


## Ablation study: Turn off hypernymy regularization

Investigating the effect of the hierarachy (PeTaL/taxonomy.txt).

In [ ]:
# note: --reg 0 turns of hypernymy regularization
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/PeTaL.yaml --model-cnf configure/models/MATCH-PeTaL.yaml --mode train --reg 0
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/PeTaL.yaml --model-cnf configure/models/MATCH-PeTaL.yaml --mode eval

!python evaluation.py --results PeTaL/results/MATCH-PeTaL-labels.npy --targets PeTaL/test_labels.npy --train-labels PeTaL/train_labels.npy

[I 210624 15:11:07 main:32] Model Name: MATCH
[I 210624 15:11:07 main:35] Loading Training and Validation Set
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'chemically_break_down_inorganic_compounds', 'detox/purify', 'manage_environmental_disturbances_in_a_community', 'protect_from_fire', 'protect_from_gases', 'send_vibratory_signals'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'detox/purify', 'protect_from_gases'] will be ignored
  .format(sorted(unknown, key=str)))
[I 210624 15:11:08 main:47] Number of Labels: 124
[I 210624 15:11:08 main:48] Size of Training Set: 800
[I 210624 15:11:08 main:49] Size of Validation Set: 100
[I 210624 15:11:08 main:68] Training
[I 210624 15:11:14 models:142] SWA Initializing
[I 210624 15:11:35 models:110] 24 1024 trai

## Study: Effect of Dataset Size on MATCH Performance

| Train size |   P@1 |
|------------|-------|
|        200 | 0.324 |
|        300 | 0.424 |
|        400 | 0.441 |
|        500 | 0.547 |
|        600 | 0.534 |
|        700 | 0.555 |
|        800 | 0.627 |

In [ ]:
# Note: I fiddled with the hardcoded train-test split in PeTal/Split.py before running this.
# Probably better to add a CLI option to specify train-test split.
%cd PeTaL/
!python3 Split.py
%cd ..
!wc PeTaL/train.json

/content/drive/Shareddrives/MATCH Attempt/MATCH/PeTaL
131
/content/drive/Shareddrives/MATCH Attempt/MATCH
    800  275452 4644161 PeTaL/train.json


In [ ]:
# Slightly modified preprocess.sh

!python3 transform_data_PeTaL.py --dataset $DATASET

!python preprocess.py \
--text-path {DATASET}/train_texts.txt \
--label-path {DATASET}/train_labels.txt \
--vocab-path {DATASET}/vocab.npy \
--emb-path {DATASET}/emb_init.npy \
--w2v-model {DATASET}/{DATASET}.joint.emb \

!python preprocess.py \
--text-path {DATASET}/test_texts.txt \
--label-path {DATASET}/test_labels.txt \
--vocab-path {DATASET}/vocab.npy \

[I 210625 17:01:33 preprocess:28] Vocab Size: 26834
[I 210625 17:01:33 preprocess:30] Getting Dataset: PeTaL/train_texts.txt Max Length: 500
[I 210625 17:01:33 preprocess:32] Size of Samples: 900
[I 210625 17:01:34 preprocess:28] Vocab Size: 26834
[I 210625 17:01:34 preprocess:30] Getting Dataset: PeTaL/test_texts.txt Max Length: 500
[I 210625 17:01:34 preprocess:32] Size of Samples: 100


In [ ]:
# Slightly modified run_models.sh

!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode train --reg 1
!PYTHONFAULTHANDLER=1 python main.py --data-cnf configure/datasets/{DATASET}.yaml --model-cnf configure/models/{MODEL}-{DATASET}.yaml --mode eval

!python evaluation.py \
--results {DATASET}/results/{MODEL}-{DATASET}-labels.npy \
--targets {DATASET}/test_labels.npy \
--train-labels {DATASET}/train_labels.npy

[I 210625 17:01:38 main:32] Model Name: MATCH
[I 210625 17:01:38 main:35] Loading Training and Validation Set
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'chemically_break_down_inorganic_compounds', 'detox/purify', 'manage_environmental_disturbances_in_a_community', 'protect_from_fire', 'protect_from_gases', 'send_vibratory_signals'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['absorb_and/or_filter_solids', 'detox/purify', 'protect_from_gases'] will be ignored
  .format(sorted(unknown, key=str)))
[I 210625 17:01:38 main:47] Number of Labels: 124
[I 210625 17:01:38 main:48] Size of Training Set: 800
[I 210625 17:01:38 main:49] Size of Validation Set: 100
[I 210625 17:01:38 main:66] Number of Edges: 101
[I 210625 17:01:38 main:68] Training
[I 210625 17:01:45 models:142] SWA Initial

idea: do k-fold cross validation?

# Results of MATCH Quick Start

Running MATCH on MAG-CS dataset as described in the paper.

In [ ]:
!./preprocess.sh

[I 210617 15:44:06 preprocess:28] Vocab Size: 500000
[I 210617 15:44:06 preprocess:30] Getting Dataset: MAG/train_texts.txt Max Length: 500
tcmalloc: large alloc 2539503616 bytes == 0x558fac048000 @  0x7f91aba831e7 0x7f91a934bea1 0x7f91a93b0928 0x7f91a93b4070 0x7f91a93b45e5 0x7f91a944d40d 0x558ed8136d54 0x558ed8136a50 0x558ed81ab105 0x558ed81a54ae 0x558ed81383ea 0x558ed81aa7f0 0x558ed81a57ad 0x558ed81383ea 0x558ed81a63b5 0x558ed81a57ad 0x558ed81383ea 0x558ed81a63b5 0x558ed81a54ae 0x558ed8077e2c 0x558ed81a7bb5 0x558ed81a54ae 0x558ed8138c9f 0x558ed8138ea1 0x558ed81a7bb5 0x558ed813830a 0x558ed81a660e 0x558ed81a54ae 0x558ed8138a81 0x558ed8138ea1 0x558ed81a7bb5
[I 210617 15:46:44 preprocess:32] Size of Samples: 634874
[I 210617 15:47:41 preprocess:28] Vocab Size: 500000
[I 210617 15:47:41 preprocess:30] Getting Dataset: MAG/test_texts.txt Max Length: 500
[I 210617 15:47:56 preprocess:32] Size of Samples: 70533


In [ ]:
!./run_models.sh

[I 210617 15:48:17 main:32] Model Name: MATCH
[I 210617 15:48:17 main:35] Loading Training and Validation Set
tcmalloc: large alloc 2539503616 bytes == 0x55777a22e000 @  0x7f4bd84601e7 0x7f4bd5d28ea1 0x7f4bd5d92b75 0x7f4bd5d9370e 0x7f4bd5e2c71e 0x55775d768d54 0x55775d768a50 0x55775d7dd105 0x55775d7d74ae 0x55775d76a3ea 0x55775d7d932a 0x55775d7d74ae 0x55775d76a3ea 0x55775d7d932a 0x55775d7d74ae 0x55775d76a3ea 0x55775d7d83b5 0x55775d7d74ae 0x55775d6a9e2c 0x55775d7d9bb5 0x55775d7d74ae 0x55775d76ac9f 0x55775d76aea1 0x55775d7d9bb5 0x55775d76a30a 0x55775d7d860e 0x55775d7d74ae 0x55775d76aa81 0x55775d76aea1 0x55775d7d9bb5 0x55775d7d74ae
tcmalloc: large alloc 2257362944 bytes == 0x5578127f4000 @  0x7f4bd84601e7 0x7f4bd5d28ea1 0x7f4bd5d8d928 0x7f4bd5d8da43 0x7f4bd5ddd2d4 0x7f4bd5e1cb90 0x55775d768d54 0x55775d768a50 0x55775d7dd105 0x55775d7d77ad 0x55775d76a3ea 0x55775d7d83b5 0x55775d85aec8 0x55775d850d8e 0x55775d840b95 0x55775d777a34 0x55775d7a8cc4 0x55775d769462 0x55775d7dc715 0x55775d7d77ad 0x557